<a href="https://colab.research.google.com/github/yuta0306/MLCompetition/blob/main/fishfinder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!ls drive/MyDrive/fishfinder/datasets

 test_images	  train_annotations  'train_images 2'  'train_images 4'
'test_images 2'   train_images	     'train_images 3'  'train_images 5'


## FILE PATH

In [5]:
TRAIN_PATH = [f'drive/MyDrive/fishfinder/datasets/{path}'
                  for path in ('train_images', 'train_images 2', 'train_images 3', 'train_images 4', 'train_images 5')]
TEST_PATH = 'drive/MyDrive/fishfinder/datasets/test_images'
ANNOTATION_PATH = 'drive/MyDrive/fishfinder/datasets/train_annotations'

## Detection Object

In [6]:
classes = ('Breezer School', 'Jumper School')
num_class = len(classes) + 1
class_label = (1, 2)

## Import Libraries

In [7]:
import os
import sys
import re
from tqdm import  tqdm

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import  Adam, SGD
from torch.utils.data import TensorDataset, DataLoader
import torchvision
from torchvision import transforms

from PIL import Image

from sklearn.model_selection import StratifiedKFold, train_test_split

In [9]:
import torchvision.models.detection.faster_rcnn as faster_rcnn

## Preprocessing

In [10]:
train_path = [
              [top, path[2]]
              for top in TRAIN_PATH
              for i, path in enumerate(os.walk(top=top))
]
train_path = [f'{top}/{path}' for top, paths in train_path
                for path in paths]
test_path = [path[2] for i, path in enumerate(os.walk(top=TEST_PATH))][0]
test_path = [f'{TEST_PATH}/{path}' for path in test_path]

In [11]:
print(len(train_path))
print(len(test_path))

3387
744


In [12]:
annotation_path = [path[2] for i, path in enumerate(os.walk(top=ANNOTATION_PATH))][0]
annotation_path = [f'{ANNOTATION_PATH}/{path}' for path in annotation_path]
print(len(annotation_path))

3387


In [13]:
train_path = sorted(train_path, key=lambda x: re.split('/', x)[-1])
test_path = sorted(test_path, key=lambda x: re.split('/', x)[-1])
annotation_path = sorted(annotation_path, key=lambda x: re.split('/', x)[-1])

In [14]:
rows = [
      [image_path, json_path]
        for image_path, json_path in zip(train_path, annotation_path)
]

In [15]:
df_path = pd.DataFrame(rows, columns=['image', 'json_path'])
pd.options.display.max_colwidth = 100
df_path

,image,json_path
0,drive/MyDrive/fishfinder/datasets/train_images 5/train_0000.jpg,drive/MyDrive/fishfinder/datasets/train_annotations/train_0000.json
1,drive/MyDrive/fishfinder/datasets/train_images 5/train_0001.jpg,drive/MyDrive/fishfinder/datasets/train_annotations/train_0001.json
2,drive/MyDrive/fishfinder/datasets/train_images 5/train_0002.jpg,drive/MyDrive/fishfinder/datasets/train_annotations/train_0002.json
3,drive/MyDrive/fishfinder/datasets/train_images 5/train_0003.jpg,drive/MyDrive/fishfinder/datasets/train_annotations/train_0003.json
4,drive/MyDrive/fishfinder/datasets/train_images 5/train_0004.jpg,drive/MyDrive/fishfinder/datasets/train_annotations/train_0004.json
...,...,...
3382,drive/MyDrive/fishfinder/datasets/train_images/train_3382.jpg,drive/MyDrive/fishfinder/datasets/train_annotations/train_3382.json
3383,drive/MyDrive/fishfinder/datasets/train_images/train_3383.jpg,drive/MyDrive/fishfinder/datasets/train_annotations/train_3383.json
3384,drive/MyDrive/fishfinder/datasets/train_images/train_3384.jpg,drive/MyDrive/fishfinder/datasets/train_annotations/train_3384.json
3385,drive/MyDrive/fishfinder/datasets/train_images/train_3385.jpg,drive/MyDrive/fishfinder/datasets/train_annotations/train_3385.json


## Global Variable

In [36]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda', index=0)

In [37]:
def get_trainsform(mode='train'):
  if mode == 'train':
    transform = transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.RandomHorizontalFlip(p=5),
        transforms.ToTensor(),
    ])
  else:
    transform = transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
    ])

  return transform

## Dataset

In [82]:
class Dataset(TensorDataset):
  def __init__(self, df, mode='train', transform=None):
    self.df = df
    self.images = self.df['image'].values
    self.annotations = self.df['json_path'].values
    self.mode = mode
    self.transform = transform

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, idx):
    TOP = TRAIN_PATH if self.mode == 'train' else TEST_PATH
    image = self.images[idx]
    annotation = pd.read_json(self.annotations[idx])
    image = Image.open(image)
    if self.transform:
      image = self.transform(image)
    else:
      image = transforms.ToTensor()(image)

    if self.mode == 'test':
      return image

    labels = [
              label
              for label in annotation.index.values.tolist()
              for data in annotation.loc[label, 'labels']
              if label in classes
    ]
    labels = [
              class_label[classes.index(label)] for label in labels
    ]
    boxes = [
              data
              for label in annotation.index.values.tolist()
              for data in annotation.loc[label, 'labels']
              if label in classes
    ]
    boxes = [
             box
             for box in boxes
    ]

    if len(labels) == 0:
      labels = [0]
    if len(boxes) == 0:
      boxes = [[0, 0, 512, 512]]

    labels = torch.LongTensor(labels)
    boxes = torch.Tensor(boxes)
    target = {
        'boxes': boxes,
        'labels': labels,
    }

    return image, target

In [83]:
def collate_fn(batch):
  images, targets = list(zip(*batch))
  images = torch.stack(images)

  return images, targets

In [84]:
dataset = Dataset(df_path, 'train', transform=get_trainsform())

In [85]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, progress=True, num_classes=num_class)

In [86]:
train_set = DataLoader(dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)

## Modeling

In [87]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=.001)

In [91]:
model.to(DEVICE)
model.train()
for e in range(5):
  print(f'### Epoch: {e+1} ###')
  for i, (images, targets) in enumerate(train_set):
    images = images.to(DEVICE)
    # targets = targets.to(DEVICE)
    for target in targets:
      for k, v in target.items():
        target[k] = v.to(DEVICE)
    output = model(images, targets)
    print(output)
    
    # loss = criterion(output, targets)
    optimizer.zero_grad()
    optimizer.step()
    if i % 10 == 0:
      print(f'STEP {i} >>>')
      print(' | '.join(f'{k}: {v}' for k, v in output))

### Epoch: 1 ###


ValueError: ignored

In [81]:
images, targets

(tensor([[[[0.7882, 0.7804, 0.7412,  ..., 0.6196, 0.6235, 0.6118],
           [0.7608, 0.7882, 0.7882,  ..., 0.6118, 0.6157, 0.6196],
           [0.7765, 0.7686, 0.7882,  ..., 0.6078, 0.6118, 0.6157],
           ...,
           [0.2745, 0.2510, 0.2510,  ..., 0.2235, 0.2235, 0.2196],
           [0.2549, 0.2706, 0.2510,  ..., 0.2275, 0.2275, 0.2235],
           [0.2627, 0.2588, 0.2275,  ..., 0.2235, 0.2235, 0.2275]],
 
          [[0.9176, 0.9059, 0.8941,  ..., 0.8118, 0.8196, 0.8039],
           [0.8980, 0.9216, 0.9216,  ..., 0.8039, 0.8078, 0.8118],
           [0.9098, 0.8941, 0.9176,  ..., 0.7961, 0.8078, 0.8196],
           ...,
           [0.5451, 0.5098, 0.4902,  ..., 0.3882, 0.3882, 0.3765],
           [0.5176, 0.5176, 0.4980,  ..., 0.3843, 0.3922, 0.3843],
           [0.4941, 0.4784, 0.4627,  ..., 0.3843, 0.3882, 0.3843]],
 
          [[0.9804, 0.9804, 0.9686,  ..., 0.9608, 0.9608, 0.9490],
           [0.9804, 0.9882, 0.9804,  ..., 0.9490, 0.9451, 0.9608],
           [0.9804, 0.96